In [1]:
#default_exp snowflake.copyinto

In [2]:
#hide
from nbdev.showdoc import *
import pandas as pd

In [3]:
#export
from fastcore.basics import *
from sdsde.snowflake.query import SnowflakeConnect
from sdsde import files

import logging
import os

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

# Functions

## ``CopyInto``

In [4]:
#export


class CopyInto(SnowflakeConnect):

    def __init__(self, sfAccount: str, sfUser: str,
                 sfPswd: str, sfWarehouse: str, sfDatabase: str,
                 sfSchema: str, sfRole: str, logger=None):
        """
        Instatiation of snowflake mover class inheriting the Snowflake_
        class from utils.
        Args:
        * sfUser (str, optional): snowflake credential passed as string
        * sfPswd (str, optional): snowflake credential passed as string
        * sfWarehouse (str, optional): snowflake credential passed as string
        * sfDatabase (str, optional): snowflake credential passed as string
        * sfSchema (str, optional): snowflake credential passed as string
        * sfRole (str, optional): snowflake credential passed as string
        * logger ([type], optional): pass custom logger as many libraries are set to Warning. Defaults to None.
        """
        super().__init__(sfAccount,
                         sfUser,
                         sfPswd,
                         sfWarehouse,
                         sfDatabase,
                         sfSchema,
                         sfRole)
        store_attr()
        self._logger = logger if logger is not None else logging.getLogger(__name__)

    def insert_csv(self,
                   blob_name: str,
                   blob_path: str,
                   storage_account: str,
                   container_name: str,
                   table_name: str,
                   sas_token: str,
                   fail_on_no_insert: bool = False,
                   delimiter: str = ',',
                   encoding: str = 'UTF-8',
                   escape_unenclosed: str = None,
                   ):
        """Insert CSV function Below is how to use it

        ```python
        ci = CopyInto(sfAccount=os.environ['sfAccount'],
              sfUser=os.environ['sfUser'],
              sfPswd=os.environ['sfPswd'],
              sfWarehouse=os.environ['sfWarehouse'],
              sfDatabase=os.environ['sfDatabase'],
              sfSchema=os.environ['sfSchema'],
              sfRole=os.environ['sfRole'])
        assert ci.test_connection() == None, 'anything else the connection has failed'
        response  = ci.insert_csv(blob_name=file[0],
                                blob_path='copyinto/',
                                storage_account='os.environ['azure_account']azure_account']',
                                container_name='sdsdetesting',
                                table_name='test_insert_csv_sdsde',
                                sas_token=os.environ['DATALAKE_SAS_TOKEN_SECRET'],
                                fail_on_no_insert=False,
                                delimiter=',')
        ```

        Args:
        * blob_name (str, optional): file name in blob.
        * blob_path (str, optional): azure blob path.
        * storage_account (str, optional): azure storage account.
        * container_name (str, optional): azure container name.
        * table_name (str, optional): snowflake table name to create.
        * sas_token (str, optional): sas token for azure data lake.
        * fail_on_no_insert (bool, optional): if True then it will allow failure. Defaults to False.
        * delimiter (str, optional): file delimiter. Defaults to ','.
        * encoding (str, optional): fild encoding. Defaults to 'UTF-8'.
        * escape_unenclosed (str, optional): special file types like mta. Defaults to None.

        Returns:
        * pd.DataFrame: Response from snowflake copy into statement
        """

        # make blob name here if a path is given
        if blob_path:
            blob_path = blob_path if not blob_path.endswith('/') else blob_path[:-1]
            blob_name = blob_path + '/' + blob_name

        # read the sql file from the libary
        inserts = ['INSERT_AZURE_STORAGE_ACCOUNT_NAME_HERE',
                   'INSERT_TABLE_NAME_HERE',
                   'INSERT_CONTAINER_NAME_HERE',
                   'INSERT_FILE_NAME_HERE',
                   'INSERT_AZURE_SAS_TOKEN_HERE',
                   'INSERT_DELIMITER_HERE',
                   'INSERT_ENCODING_HERE']
        insert = [storage_account, table_name, container_name, blob_name,
                  sas_token, delimiter, encoding]
        if escape_unenclosed == 'None':
            insert.append(escape_unenclosed)
            inserts.append('INSERT_ESCAPE_UNENCLOSED_FIELD_HERE')
            sql_file = os.path.join(os.path.abspath(files.__path__[0]), 'import_data_unenclosed.sql')
        else:
            sql_file = os.path.join(os.path.abspath(files.__path__[0]), 'import_data_csv.sql')
        with open(sql_file) as file:
            query = ' '.join(file.readlines())
        for k, v in zip(inserts, insert):
            query = query.replace(k, v)
        self._logger.info(query.replace(sas_token, '**MASKED**'))
        # execute the snowflake command
        response = self.run_str_query(query)

        # output query results
        self._logger.info('snowflake insertion output:')
        self._logger.info(f'\n {response}')

        if response.loc[0].status == 'Copy executed with 0 files processed.':
            self._logger.info('No files uploaded to snowflake')
            if fail_on_no_insert:
                raise('fail_on_no_insert was equal TRUE, so program raised error')
        else:
            # check if the load was executed correctly
            assert (response['rows_parsed'][0] == response['rows_loaded'][0]), \
                "Rows loaded and parsed are not equal"

            return response

In [5]:
show_doc(CopyInto)

<h2 id="CopyInto" class="doc_header"><code>class</code> <code>CopyInto</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CopyInto</code>(**`sfAccount`**:`str`, **`sfUser`**:`str`, **`sfPswd`**:`str`, **`sfWarehouse`**:`str`, **`sfDatabase`**:`str`, **`sfSchema`**:`str`, **`sfRole`**:`str`, **`logger`**=*`None`*) :: [`SnowflakeConnect`](/sdsdesnowflake_query.html#SnowflakeConnect)

Class that holds basic snowflake functionality including testing connection
and running queries.

In [6]:
show_doc(CopyInto.insert_csv)

<h4 id="CopyInto.insert_csv" class="doc_header"><code>CopyInto.insert_csv</code><a href="__main__.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>CopyInto.insert_csv</code>(**`blob_name`**:`str`, **`blob_path`**:`str`, **`storage_account`**:`str`, **`container_name`**:`str`, **`table_name`**:`str`, **`sas_token`**:`str`, **`fail_on_no_insert`**:`bool`=*`False`*, **`delimiter`**:`str`=*`','`*, **`encoding`**:`str`=*`'UTF-8'`*, **`escape_unenclosed`**:`str`=*`None`*)

Insert CSV function Below is how to use it

```python
ci = CopyInto(sfAccount=os.environ['sfAccount'],
      sfUser=os.environ['sfUser'],
      sfPswd=os.environ['sfPswd'],
      sfWarehouse=os.environ['sfWarehouse'],
      sfDatabase=os.environ['sfDatabase'],
      sfSchema=os.environ['sfSchema'],
      sfRole=os.environ['sfRole'])
assert ci.test_connection() == None, 'anything else the connection has failed'
response  = ci.insert_csv(blob_name=file[0],
                        blob_path='copyinto/',
                        storage_account='os.environ['azure_account']azure_account']',
                        container_name='sdsdetesting',
                        table_name='test_insert_csv_sdsde',
                        sas_token=os.environ['DATALAKE_SAS_TOKEN_SECRET'],
                        fail_on_no_insert=False,
                        delimiter=',')
```

Args:
* blob_name (str, optional): file name in blob.
* blob_path (str, optional): azure blob path.
* storage_account (str, optional): azure storage account.
* container_name (str, optional): azure container name.
* table_name (str, optional): snowflake table name to create.
* sas_token (str, optional): sas token for azure data lake.
* fail_on_no_insert (bool, optional): if True then it will allow failure. Defaults to False.
* delimiter (str, optional): file delimiter. Defaults to ','.
* encoding (str, optional): fild encoding. Defaults to 'UTF-8'.
* escape_unenclosed (str, optional): special file types like mta. Defaults to None.

Returns:
* pd.DataFrame: Response from snowflake copy into statement

## ``clean_special_chars``

In [7]:
def clean_special_chars(text):
        """
        small nlp clean up tool to take odd characters that could be
        in vendor data inside of column names and then replaces empty
        spaces with ``_``

        Args:
            text (str): dataframe column names as strings

        Returns:
            str: clean column name
        """
        punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'  # noqa:
        punct += '©^®` <→°€™› ♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'  # noqa:
        for p in punct:
            text = text.replace(p, ' ')
            text = text.replace(' ', '_')
        return text

In [8]:
show_doc(clean_special_chars)

<h4 id="clean_special_chars" class="doc_header"><code>clean_special_chars</code><a href="__main__.py#L1" class="source_link" style="float:right">[source]</a></h4>

> <code>clean_special_chars</code>(**`text`**)

small nlp clean up tool to take odd characters that could be
in vendor data inside of column names and then replaces empty
spaces with ``_``

Args:
    text (str): dataframe column names as strings

Returns:
    str: clean column name

## ``create_sf_table_from_df``

In [9]:
def create_sf_table_from_df(sf, df: pd.DataFrame, table_name_sf: str, varchar: bool):
        """
        Dynamically create a table from a dataframe and
        change the dtypes to snowflake dytpes this may have
        a limitation, but can be added.

        Args:
        * df (pd.DataFrame): data frame to get dtypes
        * table_name_sf (str): snowflake table name
        * varchar: (bool, optional): this will default all dytpes to varchars if True.
        """
        select_query = f'''
            create or replace table {table_name_sf} (FEATURES_HERE);
            '''
        for k, v in dict(df.dtypes).items():
            select_query = select_query.replace('FEATURES_HERE', f'{k} {return_sf_type(str(v), varchar=varchar)}, FEATURES_HERE')
        select_query = select_query.replace(', FEATURES_HERE', '')
        logging.info(select_query)
        sf.run_str_query(select_query)

In [10]:
show_doc(create_sf_table_from_df)

<h4 id="create_sf_table_from_df" class="doc_header"><code>create_sf_table_from_df</code><a href="__main__.py#L1" class="source_link" style="float:right">[source]</a></h4>

> <code>create_sf_table_from_df</code>(**`sf`**, **`df`**:`DataFrame`, **`table_name_sf`**:`str`, **`varchar`**:`bool`)

Dynamically create a table from a dataframe and
change the dtypes to snowflake dytpes this may have
a limitation, but can be added.

Args:
* df (pd.DataFrame): data frame to get dtypes
* table_name_sf (str): snowflake table name
* varchar: (bool, optional): this will default all dytpes to varchars if True.

## ``return_sf_type``
(Adding For Test Eventually Will Move To sdsde)

In [11]:
def return_sf_type(dtype: str, varchar: bool):
        """
        simple function to convert dytpes to snowflake dtypes this
        will be come a very useful thing to have as this will dtype
        data from 10,000 rows from a dataframe, and will fail out
        on the push to snowflake. This means the vendor or the current
        SFTP needs to have a cleaning added to it to allow for data
        to make it through the pipeline. TODO: Figure out how to accepet
        a ETL clean of the data from a vendor that can be placed in place

        Args:
        * dtype (str): dtype from a df in sting form
        * varchar (bool): to default all variables to VARCHAR
        this happens due to bad vendor data and can't be resloved
        with out reading in the whole data set with low_memory=False

        Returns:
        * str: snowflake dtype
        """
        if varchar is True:
            dtype = 'VARCHAR'
        elif 'int' in dtype.lower():
            dtype = 'NUMBER'
        elif 'float' in dtype.lower():
            dtype = 'FLOAT'
        elif 'object' in dtype.lower():
            dtype = 'VARCHAR'
        elif 'bool' in dtype.lower():
            dtype = 'VARCHAR'  # TODO: Limitation found before change once resloved by sf
        elif 'date' in dtype.lower():
            dtype = 'DATETIME'  # TODO: Might break with certain datetimes most generic
        else:
            logging.error('odd dtype not seen needs to be resloved...')
            sys.exit()
        return dtype

In [12]:
show_doc(return_sf_type)

<h4 id="return_sf_type" class="doc_header"><code>return_sf_type</code><a href="__main__.py#L1" class="source_link" style="float:right">[source]</a></h4>

> <code>return_sf_type</code>(**`dtype`**:`str`, **`varchar`**:`bool`)

simple function to convert dytpes to snowflake dtypes this
will be come a very useful thing to have as this will dtype
data from 10,000 rows from a dataframe, and will fail out
on the push to snowflake. This means the vendor or the current
SFTP needs to have a cleaning added to it to allow for data
to make it through the pipeline. TODO: Figure out how to accepet
a ETL clean of the data from a vendor that can be placed in place

Args:
* dtype (str): dtype from a df in sting form
* varchar (bool): to default all variables to VARCHAR
this happens due to bad vendor data and can't be resloved
with out reading in the whole data set with low_memory=False

Returns:
* str: snowflake dtype

# How to Use

In [13]:
show_doc(CopyInto)

<h2 id="CopyInto" class="doc_header"><code>class</code> <code>CopyInto</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CopyInto</code>(**`sfAccount`**:`str`, **`sfUser`**:`str`, **`sfPswd`**:`str`, **`sfWarehouse`**:`str`, **`sfDatabase`**:`str`, **`sfSchema`**:`str`, **`sfRole`**:`str`, **`logger`**=*`None`*) :: [`SnowflakeConnect`](/sdsdesnowflake_query.html#SnowflakeConnect)

Class that holds basic snowflake functionality including testing connection
and running queries.

In [14]:
show_doc(CopyInto.insert_csv)

<h4 id="CopyInto.insert_csv" class="doc_header"><code>CopyInto.insert_csv</code><a href="__main__.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>CopyInto.insert_csv</code>(**`blob_name`**:`str`, **`blob_path`**:`str`, **`storage_account`**:`str`, **`container_name`**:`str`, **`table_name`**:`str`, **`sas_token`**:`str`, **`fail_on_no_insert`**:`bool`=*`False`*, **`delimiter`**:`str`=*`','`*, **`encoding`**:`str`=*`'UTF-8'`*, **`escape_unenclosed`**:`str`=*`None`*)

Insert CSV function Below is how to use it

```python
ci = CopyInto(sfAccount=os.environ['sfAccount'],
      sfUser=os.environ['sfUser'],
      sfPswd=os.environ['sfPswd'],
      sfWarehouse=os.environ['sfWarehouse'],
      sfDatabase=os.environ['sfDatabase'],
      sfSchema=os.environ['sfSchema'],
      sfRole=os.environ['sfRole'])
assert ci.test_connection() == None, 'anything else the connection has failed'
response  = ci.insert_csv(blob_name=file[0],
                        blob_path='copyinto/',
                        storage_account='os.environ['azure_account']azure_account']',
                        container_name='sdsdetesting',
                        table_name='test_insert_csv_sdsde',
                        sas_token=os.environ['DATALAKE_SAS_TOKEN_SECRET'],
                        fail_on_no_insert=False,
                        delimiter=',')
```

Args:
* blob_name (str, optional): file name in blob.
* blob_path (str, optional): azure blob path.
* storage_account (str, optional): azure storage account.
* container_name (str, optional): azure container name.
* table_name (str, optional): snowflake table name to create.
* sas_token (str, optional): sas token for azure data lake.
* fail_on_no_insert (bool, optional): if True then it will allow failure. Defaults to False.
* delimiter (str, optional): file delimiter. Defaults to ','.
* encoding (str, optional): fild encoding. Defaults to 'UTF-8'.
* escape_unenclosed (str, optional): special file types like mta. Defaults to None.

Returns:
* pd.DataFrame: Response from snowflake copy into statement

# Tests

Unenclosed Example for MTA for example there are certain files that will lead to issues just another example of this.

In [47]:
#skip
from sdsde.azure.filehandling import FileHandling
from sdsde.wrapper.azurewrapper import blob_pusher, blob_puller, unlink_files

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
fh = FileHandling(os.environ['connection_str'])
if fh.ls_blob(container_name='dsdetesting', path='copyinto')[0] in ['test_nielson_copy_into.csv']:
    logger.info('file already exists....')
else:
    logger.error('Container was deleted remaking example file...')
    container_name = 'digitaladls'
    file = ['nielsen/dev/TMW/Daily/Raw/_Resorts_TM_Warehouse_Feed_286019_20210830.csv']
    blob_puller(files=file,
                connection_str=os.environ['connection_str'],
                container_name=container_name,
                drop_location='.',
                overwrite=True)
    assert os.path.exists(file[0].split('/')[-1]) == True, 'blob puller failed to grab data must have been deleted from dev change to prod'
    import pandas as pd
    df = pd.read_csv('_Resorts_TM_Warehouse_Feed_286019_20210830.csv')
    df = df[:10000]
    df.head()
    df.to_csv('test_nielson_copy_into.csv', index=False)
    file = ['test_nielson_copy_into.csv']
    container_name = 'sdsdetesting'
    blob_pusher(file_path=file,
                container_name=container_name,
                blob_dest=['copyinto'],
                connection_str=os.environ['connection_str'],
                overwrite=True)
    unlink_files([file[0].split('/')[-1]])
logger.info('Pulling Copy Into File To Create Temp Snowflake Table for Test')
blob_puller(files=['copyinto/test_nielson_copy_into.csv'],
            connection_str=os.environ['connection_str'],
            container_name='sdsdetesting',
            drop_location='.',
            overwrite=True)
ci = CopyInto(sfAccount=os.environ['sfAccount'],
              sfUser=os.environ['sfUser'],
              sfPswd=os.environ['sfPswd'],
              sfWarehouse=os.environ['sfWarehouse'],
              sfDatabase=os.environ['sfDatabase'],
              sfSchema=os.environ['sfSchema'],
              sfRole=os.environ['sfRole'])
assert ci.test_connection() == None, 'anything else the connection has failed'
df = pd.read_csv('test_nielson_copy_into.csv', nrows=150, low_memory=True)
df.columns = [clean_special_chars(x) for x in df.columns]
create_sf_table_from_df(ci, df, 'test_insert_csv_sdsde', True) 
unlink_files(['test_nielson_copy_into.csv'])
connect_str = os.environ['connection_str']
response  = ci.insert_csv(blob_name='test_nielson_copy_into.csv', 
                          blob_path='copyinto/',
                          storage_account=os.environ['azure_account'],
                          container_name='sdsdetesting',
                          table_name='test_insert_csv_sdsde', 
                          sas_token=os.environ['DATALAKE_SAS_TOKEN_SECRET'],
                          fail_on_no_insert=False,
                          delimiter=',',
                          escape_unenclosed = 'None')
assert response.rows_loaded[0] == 10000, 'Insert function failed to insert the 10,000 observations'
df =  ci.run_str_query("SELECT TOP 10 * FROM test_insert_csv_sdsde;")
assert df.shape == (10, 34), 'Table Did not load properly go to 04_snowflake_copyinto'
ci.run_str_query('DROP TABLE test_insert_csv_sdsde;')

ERROR:__main__:Container was deleted remaking example file...
INFO:sdsde.azure.filehandling:nielsen/dev/TMW/Daily/Raw/_Resorts_TM_Warehouse_Feed_286019_20210830.csv to ./_Resorts_TM_Warehouse_Feed_286019_20210830.csv
INFO:sdsde.azure.filehandling:sdsdetesting is a valid
INFO:sdsde.azure.filehandling:ContainerAlreadyExists
INFO:sdsde.azure.filehandling:Uploading test_nielson_copy_into.csv, to to Azure Storage copyinto/test_nielson_copy_into.csv
INFO:sdsde.azure.filehandling:Azure Upload Complete
INFO:__main__:Pulling Copy Into File To Create Temp Snowflake Table for Test
INFO:sdsde.azure.filehandling:copyinto/test_nielson_copy_into.csv to ./test_nielson_copy_into.csv
INFO:sdsde.snowflake.query:sqlalchemy snowflake engine created
INFO:sdsde.snowflake.query:connection to snowflake successful
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:connection to snowflake successful
INFO:root:
            create or replace table test_insert_csv_sdsde (Date VARCHAR, CONV_TYPE VARCHAR

Regular File Being Tested

In [49]:
#skip
df = ci.run_str_query("SELECT * FROM machinelearningoutputs.dev.dl_ltr Limit 1000;")
df.columns = [clean_special_chars(x) for x in df.columns]
create_sf_table_from_df(ci, df, 'test_insert_csv_sdsde', True)
df.to_csv('regular_file.csv', index=False)
file = ['regular_file.csv']
container_name = 'sdsdetesting'
blob_pusher(file_path=file,
        container_name=container_name,
        blob_dest=['copyinto'],
        connection_str=os.environ['connection_str'],
        overwrite=True)
ci = CopyInto(sfAccount=os.environ['sfAccount'],
              sfUser=os.environ['sfUser'],
              sfPswd=os.environ['sfPswd'],
              sfWarehouse=os.environ['sfWarehouse'],
              sfDatabase=os.environ['sfDatabase'],
              sfSchema=os.environ['sfSchema'],
              sfRole=os.environ['sfRole'])
assert ci.test_connection() == None, 'anything else the connection has failed'
response  = ci.insert_csv(blob_name=file[0], 
                          blob_path='copyinto/',
                          storage_account=os.environ['azure_account'],
                          container_name='sdsdetesting',
                          table_name='test_insert_csv_sdsde', 
                          sas_token=os.environ['DATALAKE_SAS_TOKEN_SECRET'],
                          fail_on_no_insert=False,
                          delimiter=',')
assert response.rows_loaded[0] == 1000, 'Insert function failed to insert the 10,000 observations'
ci.run_str_query('DROP TABLE test_insert_csv_sdsde;')
unlink_files(file)

INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:root:
            create or replace table test_insert_csv_sdsde (ecid VARCHAR, likelihoodtoreturnrate VARCHAR, likelihoodtoreturnlabel VARCHAR, fiscalyear VARCHAR, modeltype VARCHAR, experiment VARCHAR, modelname VARCHAR, uploadtime VARCHAR);
            
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:Table TEST_INSERT_CSV_SDSDE successfully created.
INFO:sdsde.azure.filehandling:sdsdetesting is a valid
INFO:sdsde.azure.filehandling:ContainerAlreadyExists
INFO:sdsde.azure.filehandling:Uploading regular_file.csv, to to Azure Storage copyinto/regular_file.csv
INFO:sdsde.azure.filehandling:Azure Uplo

# Create

In [15]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_azure.ipynb.
Converted 02_utils_dataframes.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 02_utils_stfp.ipynb.
Converted 02_utils_traininghelpers.ipynb.
Converted 02_utils_traininghelpers_fastai.ipynb.
Converted 03_dstools_preparedata.ipynb.
Converted 04_snowflake_copyinto.ipynb.
Converted 04_snowflake_copyinto2.ipynb.
Converted 04_snowflake_query.ipynb.
Converted 05_azure_wrappers.ipynb.
Converted 06_modeling_inference.ipynb.
Converted 06_modeling_inference_fastai.ipynb.
Converted 06_modeling_premodel.ipynb.
Converted 06_modeling_preprocessing.ipynb.
Converted 06_modeling_preprocessing_fastai.ipynb.
Converted 06_modeling_training.ipynb.
Converted 06_modeling_training_fastai.ipynb.
Converted 07_Binary_Classification_Fastai_Example_Notebook.ipynb.
Converted index.ipynb.
